In [1]:
from bs4 import BeautifulSoup as bs
import requests
import datetime, time
import pickle

import pandas as pd
import numpy as np

import os
import sys
import io
import shutil

import matplotlib.pyplot as plt

In [2]:
# !pip install cfscrape # 403 forbidden, cloudflare error을 해결하기 위한 모듈
import cloudscraper
scraper = cloudscraper.create_scraper()

In [3]:
# # !pip install cfscrape # 403 forbidden, cloudflare error을 해결하기 위한 모듈
# import cfscrape
# scraper = cfscrape.create_scraper()
# # 이후 403 error이 발생한 곳에는 requests 대신 scraper 사용

In [4]:
headers = {'Referer': 'https://kr.investing.com/',
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
           AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36 Edg/125.0.0.0'}

In [5]:
sys.path.append('../../data/constant')
from constants import COMPANY_CODE

In [6]:
def concat_df(df_o, df, dup_col, sort_col):
    df_o = pd.concat([df_o, df], ignore_index=True)
    df_o.drop_duplicates(subset=[dup_col], keep='last', inplace=True) # dup_col 중첩제거 기준 컬럼 이름: "time", "date" 등
#     df_o.drop_duplicates(subset=[dup_col], keep='first', inplace=True)
    df_o.sort_values(by=[df_o.columns[sort_col]], inplace=True) # sort_col 정렬 기준 컬럼 번호
    df_o.index = np.arange(0, len(df_o))  # 일련 번호 오름차순으로 재 설정
    return df_o

## 시간별 시세

In [7]:
# 특정일의 시간별 시세 취득을 위한 페이지별 table 데이터 취득. page는 날짜별로 약 40쪽. 09:00:00부터 1분간격으로 15:58:00 까지.
def get_piece_time_price(url_t):
    res = scraper.get(url_t, headers=headers)
    class_name = 'type2'
    df = pd.read_html(io.StringIO(str(res.text)), attrs={"class": class_name}, flavor=["lxml", "bs4"])[0]
    
    df = df.dropna(axis=0) # delete rows with nan values
    df.columns = ['time', 'price', 'change', 'sell', 'buy', 'volume', 'change_volumn'] # rename column in english from korean

    # convert character values to integer value : 보합= 0, 하락= -, 상승= +
    df['change'] = df['change'].apply(lambda x: int(x[2:]) if x[:2] == '보합' 
                                      else (-int(x[4:].replace(',','')) if x[:2] == '하락' 
                                            else int(x[4:].replace(',',''))))  # convert characters to int
    
    df = df[['time', 'price', 'change', 'volume']]  # delete unnecessary columns

    # define variable types
    df['time'] = df['time'].apply(lambda x : datetime.datetime.strptime(x, "%H:%M").time())  # convert characters to datetime objet
    df['price'] = df['price'].astype(int)
    df['volume'] = df['volume'].astype(int)

    return df

In [8]:
# 특정일의 시간대별 가격 40쪽을 한개의 df로 묶는 기능
def get_time_price(url_base_t, code_com, collect_date):
    
    page_num = 1
    
    # make first data frame
    collect_date_time = collect_date + '160000' # 장종료후 16시 00분 00초에 시간별 시세 추출
    page = str(page_num)
    
    url = url_base_t + '?code=' + code_com + '&thistime=' + collect_date_time + '&page=' + page
    df_base = get_piece_time_price(url)
    
    page_num = page_num + 1
    
    while True:
        page = str(page_num)
        
        url = url_base_t + '?code=' + code_com + '&thistime=' + collect_date_time + '&page=' + page
        df_p = get_piece_time_price(url)
    
        df_base = concat_df(df_base, df_p, 'time', 0)  # df concat후 'time' column을 기준으로 중복제거 후 0 column을 기준으로 정렬시킴.
        # print('page_num', page_num)
        
        if df_p['time'].iloc[-1] == datetime.time(9, 00):
            break
    
        page_num = page_num + 1
    
    df_base['date'] = datetime.datetime.strptime(collect_date, '%Y%m%d') # insert column with collecting date
    df_base = df_base[['date', 'time', 'price', 'change', 'volume']]  

    return df_base

In [9]:
def get_start_end_date(): # 오늘부터 1주일 전까지 휴일, 휴장일 제외된 일자 리스트 구성. 
    end_date = datetime.datetime.today().date()
    date_list = [end_date - datetime.timedelta(days=x) for x in range(8)] # day one week (8 days) before today
    start_date = date_list[-1]
    
    datelist = pd.date_range(start_date, end_date, freq='B') # 토, 일을 제외한 주중 일자만 선택
    # df_1 = pd.DataFrame(datelist, columns=['date']) # df로 구성

    return list(datelist.map(lambda x : x.strftime('%Y%m%d')))

In [13]:
# naver_dir = 'data/naver_finance/time_data'
naver_dir = 'time_data'

url_base = 'https://finance.naver.com/item/sise_time.naver'  # sise_date

code_dic = {'005930': ['삼성전자', 'sec'], '005380': ['현대차', 'hyunmotor'], 
        '035420': ['NAVER', 'naver'], '033780': ['KT&G', 'ktng']}

code_dic = COMPANY_CODE
code_dic = {
 '034020': ['두산에너빌리티', 'doosanener'],
 '010130': ['고려아연', 'koreazinc'],
}
# code_list = list(code_dic.items())
# code_company_name = code_list[0]
# code = code_company_name[0] # 취득을 원하는 회사 주식 코드
# code_dic = {'005930': ['삼성전자', 'sec'], '005380': ['현대차', 'hyunmotor'], }
# code_dic = {'035420': ['NAVER', 'naver'], '033780': ['KT&G', 'ktng']}

c_date = ['20240619', '20240620', '20240621', '20240624', '20240625', '20240626'] # 취득이 필요한 날짜 리스트
c_date = get_start_end_date() # 취득이 필요한 날짜 리스트, naver는 오늘날짜부터 1주일전까지만 저장. c_date 시작일자는 20200101 부터 있음.
# 
# c_date = ['20240626', '20240620'] 
# c_date = ['20240619'] 
for i, (code, company_name) in enumerate(code_dic.items()):
    for collect_date in c_date:
        df_collect = get_time_price(url_base, code, collect_date)
        # add logic to save date for each date for each company
        f_name = f'{naver_dir}/{company_name[1]}_{collect_date}.csv'
        df_collect.to_csv(f_name)
        df_collect.to_pickle(f_name.replace('csv','pkl'))

    print(i+1, end=", ")

KeyboardInterrupt: 

In [11]:
c_date


['20240711', '20240712', '20240715', '20240716', '20240717', '20240718']

In [12]:
code_dic

{'005930': ['삼성전자', 'sec'],
 '373220': ['LG에너지솔루션', 'lgenergy'],
 '000660': ['SK하이닉스', 'skhynix'],
 '207940': ['삼성바이오로직스', 'ssbio'],
 '006400': ['삼성SDI', 'sdi'],
 '051910': ['LG화학', 'lgchemical'],
 '005935': ['삼성전자우', 'secpre'],
 '005380': ['현대차', 'hyunmotor'],
 '035420': ['NAVER', 'naver'],
 '000270': ['기아', 'kia'],
 '035720': ['카카오', 'kakao'],
 '005490': ['POSCO홀딩스', 'poscoholding'],
 '105560': ['KB금융', 'kbbank'],
 '028260': ['삼성물산', 'sscnt'],
 '068270': ['셀트리온', 'celltrion'],
 '012330': ['현대모비스', 'mobis'],
 '055550': ['신한지주', 'shgroup'],
 '066570': ['LG전자', 'lgelec'],
 '003670': ['포스코퓨처엠', 'poscochemical'],
 '096770': ['SK이노베이션', 'skinnovation'],
 '033780': ['KT&G', 'ktng'],
 '030200': ['KT', 'kt'],
 '003550': ['LG', 'lg'],
 '034730': ['SK', 'sk'],
 '032830': ['삼성생명', 'sslife'],
 '086790': ['하나금융지주', 'hana'],
 '009150': ['삼성전기', 'sselec'],
 '015760': ['한국전력', 'koreaelec'],
 '034020': ['두산에너빌리티', 'doosanener'],
 '010130': ['고려아연', 'koreazinc'],
 '017670': ['SK텔레콤', 'sktelecom'],
 '01

In [ ]:

{
 '034020': ['두산에너빌리티', 'doosanener'],
 '010130': ['고려아연', 'koreazinc'],
}